# Introduction
Welcome to the Pygame Modelling Workshop. You can find the track and tasks in this notebook. We assume that you have `python3.7+` with `pip` installed and you have installed the provided package `pygmodw22` with all it's dependencies in a virtual environment from which you are running this notebook. In case you have not yet done the preparatory steps, please follow the instructions [in the README file](https://github.com/mezdahun/PygameModelling22#1-prerequisites)

## Goal of the Workshop
During this workshop you will get a hands-on introduction and demonstration on modelling a typical multi-agent system using an open-source game engine (pygame). 

The main goals of this workshop is to:
1. Give an intorduction into the used flocking model
2. Give an introduction on how agent-based simulations are usually implemented and how they are implemented in the current pygame-based approach
3. Discover the provided system by real-time hands-on experimentation
4. (Optionally) look deeper in the code

# Part I.: The Three-Zone-Model

Here, we will explore an agent-based model for simulating collective dynamics of animals (e.g. fish schools) through the provided package and small tasks. Feel free to provide your answers in this notebook. Our main task is the exploration of collective dynamics in a 2D model, related to the Three-Zone-Model (Two-Zone-Models) [by Couzin et al.](https://www.sciencedirect.com/science/article/pii/S0022519302930651), where the interaction between individuals is governed by three basic interactions: long-range attraction, short-ranged repulsion and alignment. 

## Brief Introduction
### Fundamental model

The code solves a set of (stochastic) differential equation describing a set of $N$ interacting agents ($i= 1,\dots, N$). The dynamics of each agent (in 2d) is described by the following equations of motion:

$$ \frac{d \vec{r}_i}{dt}=\vec{v}_i(t) $$
$$ \vec{v}_i(t) = {s_i\cos(\varphi_i(t)) \choose s_i\sin(\varphi_i(t)) } $$
$$ \frac{d \varphi_i}{dt} = \frac{1}{s_i}\left( F_{i,\varphi} + \eta_{i,\varphi} \right) $$


Here $\vec{r}_i$, $\vec{v}_i$ are the Cartesian position and velocity  vectors of the focal agent, wth $s_i$ being the (constant) speed of agent $i$. Furthermore, $\eta_{i,\varphi}$ represents Gaussian white noise introducing randomness in the motion of individuals, and $\vec{F}_{i,\varphi}$ is the projections of the total social force inducing a turning behavior.
$$ F_{i,\varphi}=\vec{F}_i \cdot \vec{u}_{\varphi,i} = \vec{F}_i {- s_i\sin\varphi_i \choose s_i\cos\varphi_i } $$


The total effective social force is a sum of three components:
$$ \vec{F}_i=\vec{F}_{i,rep}+\vec{F}_{i,alg}+\vec{F}_{i,att} $$


**Attraction:
$$\vec{F}_{i,att}=\sum_{j \in Neigh} +\mu_{att}S_{att}({r}_{ji}) \hat{r}_{ji} $$
Repulsion:
$$\vec{F}_{i,rep}=\sum_{j \in Neigh} -\mu_{rep}S_{rep}({r}_{ji}) \hat{r}_{ji}$$
Alignment:**
$$\vec{F}_{i,alg}=\sum_{j \in Neigh} \mu_{alg}S_{alg}({r}_{ji}) (\vec{v}_j-\vec{v}_i)$$
with $\hat r = \vec{r}/|r|$.

The strength of the different interactions is set by a constant $\mu_X$ and a sigmoid function of distance, which goes from 1 to 0, with the transition point at $r_{X,0}$ and steepness $a_{X}$:
$$ S_X(r)=\frac{1}{2}\left(\tanh(-a(r-r_{X,0})+1\right) $$

<img src="data/images/scheme_ranges.png" width='800'>

**Figure2.:** Local interaction forces around an agent

<img src="data/images/int_ranges.png">

**Figure1.:** Example of the 3 interaction zones around a focal agent

# Part II.: Agent-based Simulations in pygame

[pygame](https://www.pygame.org/wiki/about) is a free, portable, highly optimized (in terms of CPU usage and multi-threading) game engine available via the main python package manager `pip`. `pygame` was originally designed to create videogames in an easy-to understand and open-source way. Many aspects and challenges of creating videogames, on the other hand, also apply to scientific agent-based simulations (e.g. parallel computing, speed optimization, highly object-oriented design). Next to providing an object-oriented and optimized framework to implement and simulate mathematical models, `pygame` also provides a unique way of interactive experimentation with the studied system via user interaction events (button press, cursor movement, etc.).

## Structure of the provided code base

To make our experimentation faster we have provided a code base including an implementation of the described Three-zone-model as a custom made python package called `pygmodw22` (**Pyg**ame **Mod**elling **W**orkshop 20**22**). The package includes a `setup.py` file that allows it to be installed via `pip` including all dependencies and version restrictions for the package. Inside the folder package `pygmodw22` you have 3 files.

1. **agent.py**: Includes the implementation of moving agents of the model as a single `Agent` class and the rules/methods of how a these agents' behavior is updated in each timestep (`update` method). Other than functionality, the `Agent` class also describes the appearance of the agents (color, radius, etc. `draw` method). Defining and drawing the agents is surpisingly easy compared to other animation methods (e.g. matplotlib animation library). This is one of the great advantages of `pygame`. Each agent class instance will include their own parameters (radius, zone parameters, etc.) hence `pygame` also makes heterogeneous system implementations easy compared to vectorized or non-object-oriented solutions.

2. **sims.py**: In this file we defined the `Simulation` class that defines the arena (in which the agents move), possible user interactions, as well as the main simulation loop. This loop will have $T$ iterations and in each timestep it will call the agents' `update` and `draw` method (among other supplementary methods).

3. **support.py**: Includes all supllementary and mathematical methods needed for the update process. These are, for example, calculation of euclidian distance, implementation of the sigmoid function, etc.

To create an example simulation, see the code below:


## Example 1.: Understanding the Simulation and Agent classes

In [1]:
# first we import the Simulation class from the code base
from pygmodw22.sims import Simulation

pygame 2.0.2 (SDL 2.0.16, Python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# now we can create a Simulation class instance as follows
simulation = Simulation(N=10,  # Number of agents in the environment
                        T=250,  # Simulation timesteps
                        width=500,  # Arena width in pixels
                        height=500,  # Arena height in pixels
                        agent_radius=10)  # Agent radius in pixels

# Let's start the main simulation loop by calling the start method of the created simulation
simulation.start()

Running simulation start method!
Creating agents!
Starting main simulation loop!
2022-06-06_11-25-38.030436 t=1
Simulation FPS: 0.0
2022-06-06_11-25-43.394949 t=100
Simulation FPS: 24.509803771972656
2022-06-06_11-25-47.483013 t=200
Simulation FPS: 24.330900192260742
2022-06-06_11-25-49.555632 Total simulation time:  11.544666


## Interactions
Pygame provides us a unique way to visualize and interact with complex systems real-time while the simulation/game is running. We implemented a few interactions that you can use throughout the workshop to better understand the system.

**Keystrokes:**

- `f`aster: increase framerate (if your system allows)
- `s`lower: decrease framerate
- `d`efault: default framerate (25fps)
- `c`olor: turn on/off coloration according to agent orientation and velocity
- `z`ones: turn on/off showing the 3 zones around the agent
- `space`: pause/continue simulation

**Cursor Events**:

- **move**: You can drag and drop agents around by clicking and holding them. This allows you to perturb the system without any need of coding.
- **rotate**: You can rotate the agents by first grabbing them and then using your mouse wheel (or scrolling event).



## Task 1: Exploring Interactions

Use the example code snippet below. Modify it to increase the simulation time to 1000 timesteps and decrease the number of agents to 3. 

After the 3 agents have aligned into the same direction pause the experiment with `space`. Rotate one of the agents by 180 degrees from it's original orientation by moving your cursor on top of it and scrolling your mouse wheel. Continue the simulation with `space`. 

1. How does the system react to your perturbation? You can also turn on coloration of the agents by their orientation (with `c`) to get an even better insight.
2. Now do the same thing again but with 10 agents. Do you see any differences?
3. Now instead of turning an agent move it further away from the group with your cursor. What do you see? How does the effect change from 2, 3 then 10 agents?


In [10]:
# now we can create a Simulation class instance as follows
simulation = Simulation(N=3,  # Number of agents in the environment
                        T=2500,  # Simulation timesteps
                        width=500,  # Arena width in pixels
                        height=500,  # Arena height in pixels
                        agent_radius=10)  # Agent radius in pixels

# Let's start the main simulation loop by calling the start method of the created simulation
simulation.start()

Running simulation start method!
Creating agents!
Starting main simulation loop!
2022-06-06_11-51-36.323961 t=1
Simulation FPS: 0.0
2022-06-06_11-51-40.877235 t=100
Simulation FPS: 24.44987678527832
2022-06-06_11-51-44.974766 t=200
Simulation FPS: 24.69135856628418
2022-06-06_11-51-49.100981 t=300
Simulation FPS: 24.390243530273438
2022-06-06_11-51-58.355662 t=400
Simulation FPS: 23.980815887451172
2022-06-06_11-52-05.633110 t=500
Simulation FPS: 23.75296974182129
2022-06-06_11-52-09.719998 t=600
Simulation FPS: 24.213075637817383
2022-06-06_11-52-13.837919 t=700
Simulation FPS: 24.15458869934082
2022-06-06_11-52-17.936544 t=800
Simulation FPS: 24.44987678527832
2022-06-06_11-52-22.041066 t=900
Simulation FPS: 24.390243530273438
2022-06-06_11-52-26.157003 t=1000
Simulation FPS: 24.390243530273438
2022-06-06_11-52-32.548942 t=1100
Simulation FPS: 24.330900192260742
2022-06-06_11-52-36.671712 t=1200
Simulation FPS: 24.15458869934082
2022-06-06_11-52-40.763154 t=1300
Simulation FPS: 24.93

SystemExit: 

## Task 2: Exploring Zones of Agents

Use the code example code snippet below. Modify it to increase the simulation time to 1000 timesteps and decrease the number of agents to only a pair of 2. 

Turn on the visualization of the local interaction zones by pressing `z`. The long-range attraction zone is denoted with a green, the intermediate alignment zone with a yellow and the short-range repulsion with a red circle around the agents. Without pausing the simulation hold one of the agents still with your cursor.  

1. How does the other agent react? Is there anything surprising or different than what you expected?
2. Why? How can you explain what you see with the effect of the 3 zones (attraction/alignment/repulsion)?
3. What happens if you also start rotating the agent at the same time you hold it? Which zone is responsible for the temporary change in the behavior? 

In [7]:
# now we can create a Simulation class instance as follows
simulation = Simulation(N=10,  # Number of agents in the environment
                        T=250,  # Simulation timesteps
                        width=500,  # Arena width in pixels
                        height=500,  # Arena height in pixels
                        agent_radius=10)  # Agent radius in pixels

# Let's start the main simulation loop by calling the start method of the created simulation
simulation.start()

Running simulation start method!
Creating agents!
Starting main simulation loop!
2022-06-06_11-39-07.432504 t=1
Simulation FPS: 0.0
2022-06-06_11-39-15.203828 t=100
Simulation FPS: 24.213075637817383
2022-06-06_11-39-19.330444 t=200
Simulation FPS: 24.213075637817383
2022-06-06_11-39-23.443120 t=300
Simulation FPS: 24.213075637817383
2022-06-06_11-39-27.552105 t=400
Simulation FPS: 24.330900192260742
2022-06-06_11-39-31.693219 t=500
Simulation FPS: 24.509803771972656
2022-06-06_11-39-35.826749 t=600
Simulation FPS: 23.980815887451172
2022-06-06_11-39-39.911073 t=700
Simulation FPS: 24.509803771972656
2022-06-06_11-39-43.984469 t=800
Simulation FPS: 24.570024490356445
2022-06-06_11-39-48.071110 t=900
Simulation FPS: 24.509803771972656
2022-06-06_11-39-52.167941 t=1000
Simulation FPS: 24.330900192260742
2022-06-06_11-39-52.204320 Total simulation time:  44.775661


## Example 2: Flocking Parameters

As we have seen in the introduction to the zonal model each agent has 3 zones of local interaction parametrized by their interaction ranges, stepness parameters and interaction strengths.

In the next section we will see how changing these parameters will influence the collective behavior of the system.

When we create a `simulation` object it has N `Agent` type objects in it's `agents` class attribute which is the list of agents in the environment. They by default have some flocking parameters such as interaction strengths. To override these default values and set our desired parameters you can do the following: 

In [19]:
from pygmodw22.sims import Simulation

# We create a Simulation class instance with it's default agents as before
simulation = Simulation(N=10,  # Number of agents in the environment
                        T=2500,  # Simulation timesteps
                        width=500,  # Arena width in pixels
                        height=500,  # Arena height in pixels
                        agent_radius=10)  # Agent radius in pixels

# we loop through all the agents of the created simulation
for agent in simulation.agents:
    print(f"Setting parameters for agent {agent.id}")
    
    # changing their default flocking parameters
    agent.s_att = 0.02  # attraction strength (AU)
    agent.s_rep = 5  # repulsion strength (AU)
    agent.s_alg = 8  # alignment strength (AU)

    agent.r_att = 250  # attraction range (px)
    agent.r_rep = 50  # repulsion range (px)
    agent.r_alg = 150  # alignment range (px)
    
    agent.steepness_att = -0.5  # steepness in attraction force calculation (sigmoid)
    agent.steepness_rep = -0.5  # steepness in repulsion force calculation (sigmoid)
    agent.steepness_alg = -0.5  # steepness in alignment force calculation (sigmoid)
    
    agent.v_max = 1
    
    
# Now we can start the simulation with the changed agents
simulation.start()

Setting parameters for agent 0
Setting parameters for agent 1
Setting parameters for agent 2
Setting parameters for agent 3
Setting parameters for agent 4
Setting parameters for agent 5
Setting parameters for agent 6
Setting parameters for agent 7
Setting parameters for agent 8
Setting parameters for agent 9
Running simulation start method!
Starting main simulation loop!
2022-06-06_12-34-27.163993 t=1
Simulation FPS: 0.0
2022-06-06_12-34-32.514173 t=100
Simulation FPS: 24.69135856628418
2022-06-06_12-34-36.616853 t=200
Simulation FPS: 24.570024490356445
2022-06-06_12-34-40.707206 t=300
Simulation FPS: 24.44987678527832
2022-06-06_12-34-44.838310 t=400
Simulation FPS: 24.15458869934082
2022-06-06_12-34-48.963990 t=500
Simulation FPS: 24.390243530273438
2022-06-06_12-34-53.064069 t=600
Simulation FPS: 24.570024490356445
2022-06-06_12-34-57.147413 t=700
Simulation FPS: 24.509803771972656
2022-06-06_12-35-01.254321 t=800
Simulation FPS: 24.509803771972656
2022-06-06_12-35-05.351827 t=900
S

SystemExit: 

## Task 3: Exploring Flocking Parameters

Use the example code snippet above. 

1. Turn off repulsion and alignment forces and leave only the attraction force intact. What do you observe? What is the reason of the behavior?
2. Turn on "Obstacle avoidance" by re-introducing the repulsion force. What has changed compared to the previous case?
3. Re-intorduce the alignment force. What is the purpose of the alignment zone, and how does it change the collective behavior?